In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from scipy.cluster import hierarchy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier as knn
import graphviz
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler

%run spotifyid.ipynb

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Shows all the rows in all dataframes for make it easy for us to interpret the data when needed
pd.set_option('display.max_rows', None)

Exception: File `'spotifyid.ipynb.py'` not found.

In [ ]:
playlist_id=['0pa8KiIgQTFEtjJav6Ff6V','4h6xWeM4scivbL50bWnnZr',
            '1CePBpYNwRCepOXyyavY5G','6xtrjdlqoxerdY1qwMHZIF',
            '01uTFdq3G0PbFvE4JIVN5d','02Bg2havRnk0iHBpMg6alV',
            '3Pof9oBfkSaB1rcF4FgH6n','1uM5YSTQhzM1rz5kyGYdnl',
            '2XBusKFVE2RxdXk4vS1BbX','67yMmf3QuGUuBPd8YS6I3C',
            '2AQTvUjTN76igRPTgSYknp','5uQtT3Nek57pDk33TEV2Ui',
            '5bpUZJ42je385zdw18IKSI','5VP0JiOi2mls1ni1Q9qHEZ',
            '5NnpIERkeDK0JmYnhSsL7N','3uN1VDvtpZw9AyGuW1Nh1p']

df = []


for idid in playlist_id:
    results = sp.playlist(idid)
    
    playlist_name = results['name']
    
    id_list=[]

    for item in results['tracks']['items']:
        try:
            track = item['track']['id']
            id_list.append(track)
        except:
            continue

    song_meta={
        'id':[],
        'album':[], 
        'name':[], 
        'artist':[],
        'explicit':[],
        'popularity':[]
    }

    for song_id in id_list:
        # get song's meta data
        meta = sp.track(song_id)

        # song id
        song_meta['id'].append(song_id)

        # album name
        album=meta['album']['name']
        song_meta['album']+=[album]

        # song name
        song=meta['name']
        song_meta['name']+=[song]

        # artists name
        s = ', '
        artist=s.join([singer_name['name'] for singer_name in meta['artists']])
        song_meta['artist']+=[artist]

        # explicit: lyrics could be considered offensive or unsuitable for children
        explicit=meta['explicit']
        song_meta['explicit'].append(explicit)

        # song popularity
        popularity=meta['popularity']
        song_meta['popularity'].append(popularity)
        

    song_meta_df=pd.DataFrame.from_dict(song_meta)

    # check the song feature
    features = sp.audio_features(song_meta['id'])
    # change dictionary to dataframe
    features_df=pd.DataFrame.from_dict(features)

    # convert milliseconds to mins
    # duration_ms: The duration of the track in milliseconds.
    # 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
    features_df['duration_ms']=features_df['duration_ms']/60000

    # combine two dataframe
    final_df=song_meta_df.merge(features_df)
    final_df['playlist_name'] = playlist_name
    
    df.append(final_df)
    

In [ ]:
df_agg = pd.concat(df)
df_agg

In [ ]:
df_agg['playlist_name'].value_counts() 

In [ ]:
# Create dummy variable for 'explicit' column

df_dummies = pd.get_dummies(df_agg, columns = ['explicit'])

# Rename duration from milleseconds to seconds 

df_dummies.rename(columns = {'duration_ms': 'duration_s'}, inplace = True)

df_dummies.head()

In [ ]:
# Saving the name of 2022 songs for future references

songs_2022 = df_dummies.loc[:,'name':'artist'].iloc[1000:]

#Drop 'id', 'uri', 'track_href', 'analysis_url', 'explicit_False' columns

df_dummies.drop(['id','uri','track_href','analysis_url','explicit_False','album', 'name', 'artist','type'], inplace=True, axis=1)
display(df_dummies)

In [ ]:
# Split into two data frames - one for the model, the other for the prediction data 

model_data = df_dummies.iloc[:999]
predict_data = df_dummies.iloc[1000:]


In [ ]:
# Create a column indicating whether the track is a Top 50 song or not

model_data['top_50'] = pd.np.where(model_data['playlist_name'].str.contains("Not"), 0,1)
display(model_data)


In [ ]:
#Drop 'playlist_name' column

model_data.drop(['playlist_name'], inplace=True, axis=1)
predict_data.drop(['playlist_name'], inplace=True, axis=1)


df_dummies --> cleaned data with top 50, not top 50, and test songs (in total 1100 songs)
model_data --> includes top 50 and not top 50 (in total 1000 songs)
predict_data --> includes tests songs we will test on prediction model data (in total 100 songs)

#excel

#model_data.to_excel("spotify.xlsx")

## Clustering

In [ ]:
#we identify the top 50 songs from the complete dataframe
top50playlists = model_data.iloc[:499]
print(top50playlists)

When your data has different values, and even different measurement units, it can be difficult to compare them. What is kilograms compared to meters? Or altitude compared to time?
The answer to this problem is *scaling*. **We can scale data into new values that are easier to compare.**

In [ ]:
#scale the data
#Scale all dataframe values (in the green colored brackets)
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

df = top50playlists

X = df[['popularity', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_s']]

scaletop50 = scale.fit_transform(X)

print(scaletop50)

In [ ]:
#Importing KMeans
from sklearn.cluster import KMeans

#elbow method to determine the correct number of clusters
distortions = []
K = range(1,20)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(scaletop50)
    distortions.append(kmeanModel.inertia_)


plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

We use 6 clusters because the elbow's steepness flattens from that point onwards. In other words, for every cluster added after the 6th, the similarity is higher.

In [ ]:
#Modeling
kmeans = KMeans(8) #k=8 means that we want 8 clusters of data
kfit = kmeans.fit(scaletop50)
identified_clusters = kfit.predict(scaletop50)
print(kfit)
print(identified_clusters)

In [ ]:
#I attach the clustering based on scaled data to the actual dataset (new column added)
top50playlists['cluster'] = identified_clusters
display(top50playlists)
#IN THE DATAFRAME, THE LAST COLUMN 'CLUSTERS' IS INCORPORATED.

## Predictions

In [ ]:
# We will work with all features but 'popularity'

predict_data_new=predict_data.loc[:, predict_data.columns != 'popularity']
display(predict_data_new)

In [ ]:
# Prediction model

# Dictionary for accuracy scores
score_dict = {
    'knn' : [],
    'randomforest' : [],
    'boost' : [],
    'svm' : [],
    'decisiontree' : [], 
    'logisticregression' : [],
    'bag' : [],
    'passiveaggressive' : [],
    'radiusneighbors' : []
}

predict_dict = {
    'knn' : [],
    'randomforest' : [],
    'boost' : [],
    'svm' : [],
    'decisiontree' : [], 
    'logisticregression' : [],
    'bag' : [],
    'passiveaggressive' : [],
    'radiusneighbors' : []
}

importance_dict = {
    'randomforest' : [],
    'boost' : [],
    'decisiontree' : [], 
}

# Iterate multiple times
n_iter = 100

def evaluate_top(data_features, model, test_size = 0.2, normalize_cf = False, prediction_data = None):
    for i in range(n_iter):
        if model not in  ['knn','randomforest','boost','svm','decisiontree','logisticregression', 'bag', 'passiveaggressive','radiusneighbors']:
            raise ValueError('Model {} not recognized. Model must be one of the following: {}'.format(model, ['knn','randomforest','boost','svm','decisiontree', 'logisticregression', 'bag', 'passiveaggressive','radiusneighbors']))
     
        #Split the data
        data_features = model_data.loc[:,'danceability':'explicit_True']
        data_target = model_data['top_50']
        x_train, x_test, y_train, y_test = train_test_split(data_features,data_target, test_size = 0.2)

        # Scale the data with minmax scalers | define min max scaler
        scaler = MinMaxScaler()
        # transform data
        x_train_scaled = scaler.fit_transform(x_train)

        init_model = None

        if model == 'knn':
            init_model = knn(n_neighbors = 1)
        elif model == 'randomforest':
            init_model = RandomForestClassifier()
        elif model == 'boost':
            init_model = GradientBoostingClassifier()
        elif model == 'svm': 
            init_model = SVC()
        elif model == 'decisiontree': 
            init_model = DecisionTreeClassifier()
        elif model == 'logisticregression':
            init_model = LogisticRegression()
        elif model == 'bag':
            init_model = BaggingClassifier()
        elif model == 'passiveaggressive':
            init_model = PassiveAggressiveClassifier()
        elif model == 'radiusneighbors':
            init_model = RadiusNeighborsClassifier(outlier_label = "most_frequent")


        fitted_model = init_model.fit(x_train_scaled ,y_train)
        x_test_scaled = scaler.transform(x_test)
        test_predictions = fitted_model.predict(x_test_scaled)
        accuracy_score = fitted_model.score(x_test_scaled,y_test)
        
        # Accuracy Dictionary
        score_dict[model].append(accuracy_score)
        
        # Feature importance Dictionary
        if model in  ['randomforest','boost','decisiontree']:
            feature_importance=fitted_model.feature_importances_
            importance_dict[model].append(feature_importance)

        cm = confusion_matrix(y_test, test_predictions, labels=fitted_model.classes_)
        
        # New data (2022 songs) scaled & predictions
        if isinstance(prediction_data, type(None)) == False:    
            predict_data_scaled = scaler.transform(predict_data_new)
            predict_data_prediction = fitted_model.predict(predict_data_scaled)
            predict_dict[model].append(predict_data_prediction) 
        
        # NOTE: I'M NOT PRINTING THE CONFUSION MATRIX BECAUSE WE WOULD HAVE 100 FOR EACH MODEL, WHAT DO YOU THINK?
        #disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=fitted_model.classes_)
        #disp.plot(cmap='cividis')
        #plt.title('Confusion Matrix of Observation Counts for {}'.format(model))
        #plt.show()

In [ ]:
# Evaluate models

evaluate_top(model_data, model='decisiontree', prediction_data = predict_data_new)
evaluate_top(model_data, model='logisticregression', prediction_data = predict_data_new)
evaluate_top(model_data, model='knn', prediction_data = predict_data_new)
evaluate_top(model_data, model='randomforest', prediction_data = predict_data_new)
evaluate_top(model_data, model='boost', prediction_data = predict_data_new)
evaluate_top(model_data, model='svm', prediction_data = predict_data_new)
evaluate_top(model_data, model='bag', prediction_data = predict_data_new)
evaluate_top(model_data, model='passiveaggressive', prediction_data = predict_data_new)
evaluate_top(model_data, model='radiusneighbors', prediction_data = predict_data_new)

In [ ]:
# Accuracy from the 100 iterations

model=['knn','randomforest','boost','svm','decisiontree','logisticregression', 'bag', 'passiveaggressive','radiusneighbors']
interval=[]
mean_accuracy=[]
for i in model:
    #Sort accuracy and select scores for the ~95% interval
    sort=sorted(score_dict[i])
    intervals=sort[2],sort[98]
    interval.append(intervals)

    #Mean accuracy for the 100 iterations
    mean_acc = np.mean(score_dict[i])
    mean_accuracy.append(mean_acc)

accuracy = [f'{i*100:.2f}%' for i in mean_accuracy]
CI_df = pd.DataFrame({'Model' : model, '95% CI' : interval, 'Mean accuracy':accuracy})
display(CI_df)

In [ ]:
# Predictions with the model with the best accuracy: Random forest (1 Yes, 0 No)

predictions=predict_dict['randomforest'][0]
predictiondf = pd.DataFrame({'2022_Songs':songs_2022['name'],'Artist':songs_2022['artist'], 'Top50?':predictions})
display(predictiondf)

In [ ]:
# NOTE DISPLAYING DATAFRAME IN CASE IS NEEDED FOR INTERPRETATIONS, IF NOT I RECOMMEND NOT DISPLAYING IT
# Feature Relevance

attributes=predict_data_new.columns.values.tolist()
importance=importance_dict['randomforest'][2]
pd.to_numeric(importance)
importancedf = pd.DataFrame({'Features':attributes,'Relevance':importance})
df_sorted_desc= importancedf.sort_values('Relevance',ascending=False)
display(df_sorted_desc)

In [ ]:
# Plot Feature relevance

df_sorted_desc.plot.bar(x="Features", y="Relevance", title='Feature Relevance')
plt.gca().set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()]) 
plt.show()